In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
import pymysql
from datetime import datetime

In [2]:
# MySQL Connection 연결 (DB 연결)
def setting_db():
    conn = pymysql.connect(host="localhost", 
                       port=3306, user="root", 
                       passwd="qlqtmajrwk",
                       db="dg",
                       charset="utf8")
    cursor = conn.cursor(pymysql.cursors.DictCursor) # 리스트로 받기(tuple은 conn.cursor())
    
    return conn, cursor

In [3]:
# stockDB 만들기
conn, cursor = setting_db()
sql = '''CREATE TABLE stockDB (datecd varchar(45) NOT NULL,
                             crp_cd varchar(8), 
                             date date,
                             open float,
                             high float,
                             low float,
                             close float,
                             diff float,
                             volume float,
                             PRIMARY KEY (datecd)
                            );
                            '''
cursor.execute(sql)
conn.close()

In [4]:
code_df = pd.read_html('http://kind.krx.co.kr/corpgeneral/corpList.do?method=download&searchType=13', header=0)[0]
code_df.종목코드 = code_df.종목코드.map('{:06d}'.format)
code_df = code_df[['회사명', '종목코드']] 
code_df = code_df.rename(columns={'회사명': 'name', '종목코드': 'code'})
code = code_df.values.tolist()

newcode = [i[1] for i in code]

In [5]:
conn, cursor = setting_db()
sql = "select crp_cd from stockDB"
cursor.execute(sql)
crp_cd = cursor.fetchall()
conn.close()

oldcode = [i['crp_cd'] for i in crp_cd]

In [6]:
changecode = set(newcode) - set(oldcode)
changecode = list(changecode)

In [7]:
urllist = []
for i in range(0, len(changecode)):
    url = 'http://finance.naver.com/item/sise_day.nhn?code=' + changecode[i]
    urllist.append(url)

In [8]:
lastpage = []
for i in range(0,len(urllist)):
    req = requests.get(urllist[i])
    html = req.text
    soup = BeautifulSoup(html, 'lxml')
    soupa = str(soup.findAll('a')[-1])
    comp = re.compile('page=.*"')
    lp = comp.findall(soupa)
    lastpage.append(lp[0][5:-1])

In [ ]:
for i in range(0, len(urllist)):
    stockdata = pd.DataFrame()
    for j in range(0, int(lastpage[i])): 
        page = j + 1
        urlpage = urllist[i] + '&page=' + '{page}'.format(page=page) 
        stockdata = stockdata.append(pd.read_html(urlpage, header=0)[0], ignore_index=True)
    stockdata = stockdata.dropna()
    stockdata["crp_cd"] = changecode[i]
    stockdata['날짜'] = pd.to_datetime(stockdata['날짜']).apply(lambda x: x.strftime('%Y-%m-%d'))
    stockdata['datecd'] = pd.to_datetime(stockdata['날짜']).apply(lambda x: x.strftime('%Y%m%d')) + changecode[i]
    stocksql = stockdata.values.tolist()
    
    conn, cursor = setting_db()
    sql = "insert into stockDB(date, close, diff, open, high, low, volume, crp_cd, datecd) values (%s, %s, %s, %s, %s, %s, %s, %s, %s)"
    cursor.executemany(sql, stocksql)
    conn.commit()
    conn.close()
    print(i)

0
